<a href="https://colab.research.google.com/github/AdamLeebubu/web3_dashboard/blob/main/MPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code above is modified from https://www.geeksforgeeks.org/implementing-shamirs-secret-sharing-scheme-in-python/#

In [ ]:
import random
from math import ceil
from decimal import Decimal
import sys

In [ ]:
def reconstruct_secret(shares):
    """
    Combines individual shares (points on graph)
    using Lagranges interpolation.

    shares is a list of points (x, y) belonging to a
    polynomial with a constant of our key.
    """
    sums = 0
    prod_arr = []

    for j, share_j in enumerate(shares):
        xj, yj = share_j
        prod = Decimal(1)

        for i, share_i in enumerate(shares):
            xi, _ = share_i
            if i != j:
                prod *= Decimal(Decimal(xi)/(xi-xj))

        prod *= yj
        sums += Decimal(prod)

    return int(round(Decimal(sums), 0))


def polynom(x, coefficients):
    """
    This generates a single point on the graph of given polynomial
    in x. The polynomial is given by the list of coefficients.
    """
    point = 0
    # Loop through reversed list, so that indices from enumerate match the
    # actual coefficient indices
    for coefficient_index, coefficient_value in enumerate(coefficients[::-1]):
        point += x ** coefficient_index * coefficient_value
    return point


def coeff(t, secret):
    """
    Randomly generate a list of coefficients for a polynomial with
    degree of t - 1, whose constant is secret.
    """
    coeff = [random.randrange(-sys.maxsize - 1, sys.maxsize) for _ in range(t - 1)]
    coeff.append(secret)
    return coeff


def generate_shares(n, m, secrets):
    """
    Split given secret into `n` shares with minimum threshold
    of m shares to recover this `secret`, using SSS algorithm.
    """
    shares = []

    for secret in secrets:
      coefficients = coeff(m, secret)
      share = []
      x = 1
      for i in range(1, n+1):
          share.append((x, polynom(x, coefficients)))
          x += 1
      shares.append(share)

    return shares

def party_calculate(shares):
  """
    each party do the calculation on the shares they have
  """
  sub_sum = []
  for i in range(0,len(shares[0])):
    temp = 0
    for j in range(0, 7):
      # print(shares[j][i][1])
      temp += shares[j][i][1]
    sub_sum.append((i+1, temp))
  return sub_sum


In [ ]:
# (3,3) sharing scheme of data aggregation for weekly data
t, n = 3, 3
data = [1,2,3,4,5,6,7]
print(f'Original Data: {data}')

# Phase I: Generation of shares
shares = generate_shares(n, t, data)
print(f'Shares: {shares}') # the party i receive shares whose first value in tuple is i

# Phase II: Secret Reconstruction
# Picking t shares randomly for reconstruction
sub_sum = party_calculate(shares)
print(f'Sub_sum: {sub_sum}')
print(f'Reconstructed result: {reconstruct_secret(sub_sum)}')
print('Desired result: 28')

Original Data: [1, 2, 3, 4, 5, 6, 7]
Shares: [[(1, 4138349628060796483), (2, 2236161516758314669), (3, -5706564333907445441)], [(1, -11152914408973768197), (2, -40175658313863442266), (3, -87068231714669022205)], [(1, -9051422315447562646), (2, -23189136038309938531), (3, -42413141168587127652)], [(1, -10611459398563223302), (2, -30403690723560658992), (3, -59376693974992307066)], [(1, -8489895495152540713), (2, -18952746832740926623), (3, -31388554012765157725)], [(1, -5515458012828864116), (2, -9747013105588909958), (3, -12694665278280137520)], [(1, -16985155340848670341), (2, -51688956678296315231), (3, -104111404012342934663)]]
Sub_sum: [(1, -57667955343753832832), (2, -171921040175601876932), (3, -342759254495544132272)]
Reconstructed result: 28
Desired result: 28
